In [18]:
import torch
import torch.nn as nn

# 获取所有可用的 GPU
gpus = torch.cuda.device_count()
print('Number of GPUs available:', gpus)

if gpus > 0:
    gpu_num = 0  # 要使用的 GPU 编号

    # 设置默认的 GPU
    torch.cuda.set_device(gpu_num)
    print('Only GPU number', gpu_num, 'used.')

    # 设置 GPU 内存增长模式
    device = torch.device(f'cuda:{gpu_num}')
    print(device)
    torch.cuda.empty_cache()
    try:
        torch.cuda.memory_allocated = 0
        torch.cuda.memory_reserved = 0
    except RuntimeError as e:
        print(e)


Number of GPUs available: 1
Only GPU number 0 used.
cuda:0


In [19]:
class BinarySource(nn.Module):
    """
    Layer generating random binary tensors.

    Parameters
    ----------
    dtype : torch.dtype
        Defines the output datatype of the layer.
        Defaults to `torch.float32`.

    seed : int or None
        Set the seed for the random generator used to generate the bits.
        Set to `None` for random initialization of the RNG.

    device : torch.device or None
        The device on which the tensor will be created.
        Defaults to `None`, which means the tensor will be created on the CPU.

    Input
    -----
    shape : 1D tensor/array/list, int
        The desired shape of the output tensor.

    Output
    ------
    : ``shape``, ``dtype``
        Tensor filled with random binary values.
    """
    def __init__(self, dtype=torch.float32, seed=None, device=None, **kwargs):
        super().__init__(**kwargs)
        self.dtype = dtype
        self._seed = seed
        self.device = device if device is not None else torch.device('cpu')
        self._rng = torch.Generator(device=self.device)
        if self._seed is not None:
            self._rng.manual_seed(self._seed)

    def forward(self, inputs):
        if self._seed is not None:
            return torch.randint(0, 2, inputs, generator=self._rng, dtype=torch.int32, device=self.device).to(self.dtype)
        else:
            return torch.randint(0, 2, inputs, dtype=torch.int32, device=self.device).to(self.dtype)


In [20]:
# 示例用法
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
binary_source = BinarySource(dtype=torch.float32, seed=42, device=device)
output = binary_source(torch.Size([10, 5]))
print(output)

tensor([[1., 1., 0., 0., 1.],
        [1., 0., 0., 0., 1.],
        [1., 1., 1., 0., 1.],
        [0., 0., 1., 0., 1.],
        [1., 0., 1., 1., 1.],
        [1., 0., 0., 1., 0.],
        [1., 1., 0., 0., 1.],
        [0., 0., 1., 1., 0.],
        [1., 0., 1., 1., 0.],
        [1., 1., 0., 0., 1.]], device='cuda:0')
